Version 0.0.1




In [ ]:
# dont run if you are in top level
import os
os.chdir("..")

# Datenformat
----------------------

Das im Weiteren verwendete Datenformat aus dem alle notwendigen Größen genommen werden ist ein modifiziertes .CSV Format. Der Vorteil dieses Formates ist, dass die Studenten immer und zu jeder Zeit die Daten so wie sie auch ein Mensch lesen würde betrachten können. 

Die Trennung einzelner Variablen in einem Event geschieht mithilfe von ";". Die Einträge der einzelnen leptonen innerhalb eines Events werden dagegen klassisch mit "," getrennt.  Damit entsteht der Vorteil individueller Anzahl an Leptonen in einem Event ohne die Einführung von zusätzlichen Platzhaltern.

In [ ]:
%matplotlib inline
#%matplotlib qt
import numpy as np
import pandas as pd

from IPython.display import display
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 50)

name_1 = "./data/mc_init/MC_2012_H_to_ZZ_to_4L_to_2el2mu_8TeV_init.csv"
name_2 = "./data/mc_init/MC_2012_ZZ_to_4L_to_4mu_8TeV_init.csv"

dataframe_1 = pd.read_csv(name_1, delimiter=";")
dataframe_2 = pd.read_csv(name_2, delimiter=";")

In [ ]:
dataframe_1

In [ ]:
dataframe_2

Für die Bearbeitung einzelner Größen können die jeweiligen Elemente aus dem str Datenformat wieder zurück in einer Liste umgewandelt werden:
Hierzu stehen die Optionen offen entweder die Bibliothek ast

In [ ]:
import ast
px = ast.literal_eval(f"[{dataframe_2.loc[3, 'px']}]")
px = np.array(px, dtype=float)
px

oder die split Methode für die Strings zu vernwenden

In [ ]:
py = dataframe_2.loc[3, 'py'].split(",")
py = np.array(py, dtype=float)
py

Daraus können neue Größen wie der transversale Impuls bestimmt werden anhand dessen eine Eingrenzung in den Größen getroffen werden soll

In [ ]:
pt = np.sqrt(px ** 2 + py ** 2)
pt

Für Myonen muss ein Minimalwert von 5 GeV für den transversalimpuls überschritten werden. Für Transversalimpulswerte darunter steigt die Wahrscheinlich einer Missidentifikation der Myonen an.
Insofern müssen alle Myonen in diesem Ereignis die die Bedingung nicht erfüllen entfernt werden.

In [ ]:
pt_minimum_filter = pt > 5
pt_minimum_filter

In [ ]:
pt = pt[pt_minimum_filter]
pt

# Anwendung der Filter mit Hilfe von Apply
----------------------

Die Anwendung dieses Filters erfolgt auf alle Variablen innerhalb dieses Ereignisses und ist für das nächste Ereignis verschieden.
Die reihenweise Anwendung des Filters auf den Datensatz ist bereits implementiert und über die Apply Klasse anwendbar.

In [ ]:
from include.processing.Apply import Apply

Bei der Erstellung des Objekts muss zusätzlich eine "allowed" und eine "calculation" Klasse übergeben werden. Die bei der Anwendung der Filter notwendigen Größen oder Filter werden anhand dieser Klassen berechnet.
Für den Beispiel des minimalen Transversalimpulses lassen sich die beiden Klassen als eine Ansammlung von Funktionen darstellen, die alleine stehen könnten (@staticmethod), aber logisch in einer Klasse zusammengefasst werden.

In [ ]:
class Calc_Start(object):
    
    @staticmethod
    def pt(px, py):
        return np.sqrt(px ** 2 + py ** 2)

class Allowed_Start(object):
    
    @staticmethod
    def min_pt(pt, look_for):
        if look_for == "electron":
            return pt > 7.0
        if look_for == "muon":
            return pt > 5.0

In [ ]:
name_2

In [ ]:
process = Apply(input_=name_2, particle_type="muon", use_n_rows=10000,
                calc_instance=Calc_Start, allowed_instance=Allowed_Start)

In einem zwischenschritt, der in dem Filter "check_min_pt" schon enthalten ist lässt sich der Transversalimpuls separat hinzufügen:

In [ ]:
process.add_variable("pt")

In [ ]:
process.data

Die Häufigkeitsverteilung lässt sich wie folgt visualisieren:

In [ ]:
process.hist_of_variable(variable="pt", bins=100, hist_range=(0, 80))

Die Anwendung des Filters für den minimalen Transversalimpuls und die Entfernung aller nachfolgend entstandenen Ereignisse, die weniger als vier Leptonen enthalten liefert die entsprechende Verteilung:

In [ ]:
process.filter(filter_name="check_min_pt")
process.hist_of_variable(variable="pt", bins=100, hist_range=(0, 80))

Ebenso lassen sich auch andere Variablen durch die Häufigkeitsverteilungen visualisieren.
Der durchgeführte Cut beeinflusst hauptsächlich die Transversalimpulsverteilung, ändert an der Verteilung der Pseudorapidität aber dagegen wenig

In [ ]:
# place to play

# Wichtig ist sicherzustellen, dass die für die Analyse getroffenen VOrraussetzungen sichergestellt werden.
# Am einfachsten lässt es esich durch die Betrachtung der Häufigkeitsverteilungen darstellen um beispielsweise
# zu gewährleisten das das ZZ Bosonenpaar aus einem virtuellen und einem rellen Z Boson besteht.

# Zusatz: Findet Barad-dûr

# Berechnung wichtiger Größen
----------------------

Dem Beispiel des Transversalimpulses folgend sollen alle nachfolgenden Größen, die berechnet werden müssen von den Studenten implementiert werden. Funktionen die komplizierter sind werden über die Vererbung vorgeladen.

In [ ]:
from include.processing.CalcAndAllowerInit import CalcInit

In [ ]:
class CalcStudent(Calc_Start, CalcInit):
    '''
    Class for the calculation of certain sizes that are used for
    the cuts or are essential for the reconstruction.
    '''

    @staticmethod
    def combined_charge(charge, combine_num):
        '''
        Tests whether an electrically neutral charge combination is possible.

        :param charge: ndarray
                       1D array containing data with "int" type.
        :param combine_num: int
                            4 if look_for is not "both", 2 else
        :return: bool
        '''

    @staticmethod
    def eta(px=None, py=None, pz=None, energy=None):
        '''
        Calculates the pseudorapidity.
        Optional with or without energy.

        :param px: ndarray
                   1D array containing data with "float" type.
        :param py: ndarray
                    1D array containing data with "float" type.
        :param pz: ndarray
                   1D array containing data with "float" type.
        :param energy: ndarray
                       1D array containing data with "float" type.
        :return: ndarray
                 1D array containing data with "float" type.
        '''

    @staticmethod
    def invariant_mass_square(px, py, pz, energy=None, eta=None, phi=None):
        '''
        Calculates the square of the invariant mass.
        Optional with or without energy.
        Optionally with or without eta and phi.

        :param phi: ndarray
                    1D array containing data with "float" type.
        :param eta: ndarray
                    1D array containing data with "float" type.
        :param px: ndarray
                   1D array containing data with "float" type.
        :param py: ndarray
                   1D array containing data with "float" type.
        :param pz: ndarray
                   1D array containing data with "float" type.
        :param energy: ndarray
                       1D array containing data with "float" type.
        :return: ndarray
                 1D array containing data with "float" type.
        '''

    @staticmethod
    def phi(px, py):
        '''
        Calculation of the angle phi.

        :param px: ndarray
                   1D array containing data with "float" type.
        :param py:  ndarray
                    1D array containing data with "float" type.
        :return: ndarray
                 1D array containing data with "float" type.
        '''

    @staticmethod
    def delta_phi(phi1, phi2):
        '''
        Calculation of the difference between two phi angles.

        :param phi1: ndarray
                     1D array containing data with "float" type.
        :param phi2: ndarray
                     1D array containing data with "float" type.
        :return: ndarray
                 1D array containing data with "float" type.
        '''

    @staticmethod
    def delta_r(eta, phi):
        '''
        Calculation of delta_r.

        :param eta: ndarray
                    1D array containing data with "float" type.
        :param phi: ndarray
                    1D array containing data with "float" type.
        :return: ndarray
                 1D array containing data with "float" type.
        '''

# Erstellung der Cuts
----------------------

Analog zu dem Cut auf den minimalen Transversallimpuls lassen sich weitere Cuts einführen, die im weiteren verwendet werden sollen

In [ ]:
from include.processing.CalcAndAllowerInit import AllowedInit

In [ ]:
class AllowedStudent(Allowed_Start, AllowedInit):
    '''
    Class that introduces certain cuts and thus restricts the leptons in the events.
    '''
    
    @staticmethod
    def delta_r(delta_r):
        '''
        Checks if delta_r is smaller than the allowed value.

        :param delta_r: ndarray
                        1D array containing data with `float` type.
        :return: ndarray
                 1D array containing data with `bool` type.
        '''

    @staticmethod
    def rel_pf_iso(rel_pf_iso):
        '''
        Checks if rel_pf_iso is smaller than the allowed value.

        :param rel_pf_iso: ndarray
                           1D array containing data with `float` type.
        :return: ndarray
                 1D array containing data with `bool` type.
        '''

    @staticmethod
    def misshits(misshits):
        '''
        Checks if the minimum number of misshits was kept.

        :param misshits:
        :return:
        '''

    @staticmethod
    def pt(p_t, look_for, coll_size=4):
        '''
        Checks if the exact pedingun regarding pt is observed.
        (>20 GeV: >= 1; >10 GeV: >= 2; >Minimum pt: >= 4).

        :param p_t: ndarray
                    1D array containing data with `float` type.
        :param look_for: str
                         "muon"; "electron" or "both"
        :param coll_size: int
                          4 if look_for is not "both", 2 else
        :return: ndarray
                 1D array containing data with `bool` type.
        '''

    @staticmethod
    def eta(eta, look_for):
        '''
        Checks if the pseudorapidity of leptons is valid.

        :param eta: ndarray
                    1D array containing data with "float" type.
        :param look_for: str
                         "muon"; "electron" or "both"
        :return: ndarray
                 1D array containing data with "bool" type.
        '''

    @staticmethod
    def type(typ, look_for):
        '''
        Checks for the permitted classification of leptons.

        :param typ: ndarray
                    1D array containing data with "float" type.
        :param look_for: str
                         "muon"; "electron" or "both"
        :return: ndarray
                 1D array containing data with "bool" type.
        '''

    @staticmethod
    def impact_param(sip3d, dxy, dz):
        '''
        Checks if the impact parameters of the collision are valid and sorts out
        events that do not have a clear and equal collision point.

        :param sip3d: ndarray
                      1D array containing data with "float" type.
        :param dxy: ndarray
                    1D array containing data with "float" type.
        :param dz: ndarray
                   1D array containing data with "float" type.
        :return: ndarray
                 1D array containing data with "bool" type.
        '''

    @staticmethod
    def zz(z1, z2):
        '''
        Checks if the Z1 candidate and the Z2 candidate is within the allowed range.

        :param z1: float
        :param z2: float
        :return: bool
        '''

# Anwendung der Filter und Rekonstruktion auf MC - Simulationen
--------------------------------------------------------------------------------------------------------------

Die vorhergehende Implementierung der Cuts kann anhand der Untergrund MC - Simulatione getestet werden. Dies soll das gezielte Hinarbeiten auf ein gewisses Ziel verhindern, da durch eine geringe Anzahl an späteren Ereignissen in der eigentlichen Messung die Tatsache weitestgehend vermieden werden soll gezielt Ereignisse in einem Bereich subjektiv auzuwählen.

Die festgelegten Cuts werden dann anschließend auf die Signal MC -  Simulatione und die Messung angewendet. Hierzu werden zunächst alle MC - Simulationen in einer Liste aus tupels gesammelt.

In [ ]:
from include.processing.ApplyHelper import ProcessHelper

In [ ]:
# Alle Untergrund MC und Signal MC für m = 125 GeV
mc_files = True
# Signal MC für weitere Massen m = [115, 120, 122, 124, 128, 130, 135, 140, 145, 150]
other_mc_files = True
# Durchgeführte Messung
run_files = True

file_tuples = []
if mc_files:
    file_tuples += ProcessHelper.create_tuple("./data/mc_aftT/")    
if other_mc_files:
    import os
    list_of_other_mc = []
    for name in os.listdir("./data/other_mc/"):
        file_tuples += ProcessHelper.create_tuple(os.path.join(".", "data", "other_mc", name, "mc_init/"))
if run_files:
    file_tuples += ProcessHelper.create_tuple("./data/ru_aftT/")
    
file_tuples[0]

Mit den oben implementierten Methoden der Klasse CalcStudent und AllowedStudent lassen sich folgende operationen durchführen:

In [ ]:
Apply.help()

Die ganze Filter und Rekonstruktionsablauf kann in einer Funktion zusamengefasst werden. Die Reihenfolge der Filter ist hierbei für die Laufzeit entscheidend.

In [ ]:
def filter_and_reco_process(used_pair):
    process = Apply(input_=used_pair.name, particle_type=used_pair.particle, multi_cpu=True,
                    calc_instance=Calc, allowed_instance=Allowed)
    process.filter(filter_name="check_type",
                   quicksave=ProcessHelper.change_on_affix(used_pair.name, "aftT"))
    process.filter(filter_name="check_q")
    process.filter(filter_name="check_impact_param")
    process.filter(filter_name="check_q")
    process.filter(filter_name="check_rel_iso")
    process.filter(filter_name="check_q")
    if process.particle_type != "muon":
        process.filter(filter_name="check_misshit")
    process.filter(filter_name="check_q")
    process.filter(filter_name="check_min_pt")
    process.filter(filter_name="check_q")
    process.filter(filter_name="check_eta")
    process.filter(filter_name="check_q")
    process.filter(filter_name="check_exact_pt")
    process.filter(filter_name="check_m_2l")
    process.filter(filter_name="check_m_4l",
                   quicksave=ProcessHelper.change_on_affix(used_pair.name, "befZ"))
    process.reconstruct(reco_name="zz", 
                        quicksave=ProcessHelper.change_on_affix(used_pair.name, "aftZ"))
    process.reconstruct(reco_name="mass_4l_out_zz",
                        quicksave=ProcessHelper.change_on_affix(used_pair.name, "aftH"))
    del process

In [ ]:
from tqdm import tqdm
from IPython.display import clear_output

for pair in tqdm(file_tuples):
    filter_and_reco_process(pair)
    clear_output()

# Betrachtung der finalen Verteilungen
------------------------------------------------------------------

Abschließend besteht die Möglichkeit gewollte Verteilungen darzustellen, wobei die MC Simulation in den Signal und Untergrundteil aufgespalten wird.  
Hierzu sollte die gewünschte Masse der Signal MC-Simulation verwendet werden, die zuvor in einem Ordner mit allen gewünschten MC - Smulationen zusammengetragen werden. Mögliche Größen sind:

In [ ]:
print(ProcessHelper.print_possible_variables("./data/mc_aftH/MC_2012_ZZ_to_4L_to_2el2mu_8TeV_aftH.csv"))

An dieser Stelle ist es fraglich ob  z1_index, z2_index, sowie z1_tag bzw. z2_tag Größen sind die eine ausführlicher Betrachtung bedürfen.

In [ ]:
from include.histogramm.HistOf import HistOf
import matplotlib.pyplot as plt

%matplotlib inline
plt.rcParams["figure.figsize"] = (12, 9)

In [ ]:
h = HistOf(mc_dir="./data/mc_aftH/", ru_dir="./data/ru_aftH/")  

In [ ]:
h.variable("mass_4l", 30, (70, 181))
ax = plt.gca()
ax.set_xlabel(r"$m_{4\ell}$ in GeV")
ax.set_ylabel("Bineinträge")
plt.show()

In [ ]:
# place to play and verify things

# Parametrisierung des Untergrundes
----------------------

Für die Vermeidung von Fluktuationen ist es nun die Aufgabe die Verteilung der Untegrrund MC Simulation durch eine Wahrscheinlichkeitsdichte auszudrücken.  
Hierzu soll das einfachste Modell verwendet werden, das die besten Ergebnisse erzielt.  Die Implementierung von linearkombinationen von Legendrepolynomen wird den Studenten zur Verfügung gestellt (LLC).

In [ ]:
from include.histogramm.HistDataGetter import HistDataGetter as HDG
from include.parameterization.LegendrePoly import LegendreLinearCombination as LLC

In [ ]:
bins, hist_range, info = 30, (106, 151), [["2012"], ["A-D"]]
mc_dir, ru_dir = "./data/mc_aftH/", "./data/mc_aftH/"

Variante:

 1. <ins>Die Daten aus dem Histogramm werden roh ausgegeben.</ins>  
    Die skalierten Unsicherheiten für die Bineinträge müssen separat berechnet werden. Zudem muss die Skalierung des
    Histogramms vorgenommen werden.  
    Das Zugreifen auf die einzelnen Teile kann über dne Attribut data geschehen, wobei das ein geschachteltes PYTHON
    dict ist. Die erste Ebene enthält die drei einzelnen Kanäle. In jedem der Kanäle ist anschließend die unskalierten
    Histogrammeinträge, der Korrekturfaktor und die Daten, die für die Histogrammbildung verwendet wurden gespeichert.

In [ ]:
raw_mc_data = HDG(bins=bins, hist_range=hist_range, 
           info=info, mc_dir=mc_dir, ru_dir=ru_dir).get_mc_raw(tag="background")
print(list(raw_mc_data.data.keys()))
raw_mc_data.data["mc_track_ZZ_4el_2012"]

In [ ]:
# your functions goes here
# your fit goes here

 2. <ins>Die Daten aus dem Histogramm werden zusammengefasst ausgegeben</ins>  
     und die Unsicherheit bereits berechnet

In [ ]:
from include.fits.McFitInit import McFitInit

In [ ]:
format_data = McFitInit(bins=bins, hist_range=hist_range, tag="background", mc_dir=mc_dir, ru_dir=ru_dir)
bins, hist_range, hist_entries, combined_hist_entries_errors = format_data.variables_for_hist_fit

In [ ]:
# your functions goes here
# your fit goes here

Den Studenten steht es frei entweder die Variante 1. zu verfolgen oder die den Studenten gegebene unvollständige Klasse MCFit derart zu vervollständigen, dass sie Fits beliebiger Funktionen durchführen können. Der Vorteil wäre hierbei, dass die weitere Benutzung bei der Signalparametrisierung möglich ist.

# Auswahl der passenden  Signal MC
----------------------

Das Ziel dieses Abschnittes ist eine Eingrenzung der möglichen Signal MC Simulationen durchzuführen und sich auf ein einziges festzulegen und damit weiterarbeiten.

## Vorläufige Parametrisierung der Signal MC

Den Studenten werden unterschiedliche Signal MC Simulationen zur verfügung gestellt, aus denen eine geeignete Wahl getroffen werden soll. Um den zeitlichen Rahmen des Praktikums einhalten zu können wird außerhalb des Praktikums eine grobe Parametrisierung dieser Verteilungen durchgeführt.
Hierzu werden die einzelnen Signal MC Simulationen bei unterscheidlcihen Massen durch eine Gaussverteilung $\mathcal{G} \left( m_{4\ell}, m_{\mathrm{H}}, \sigma_{\mathrm{G}} \right)$ parametrisiert.
Das Ziel ist es die Abhängigkeit der Breite von der invarianten Masse festzustellen und durch ein passendes Modell zu beschreiben. Die Transversalimpulsauflösung des Detektors ist hierbei der größte EInfluss.  
Die Ergebnisse dieser Parametrisierungen sind in einem Pandas Dataframe zusammengefasst.

In [ ]:
import pandas as pd
df_ = pd.read_csv("./data/other_mc_param_solution/temp_sigma_mu.csv")
df_

In [ ]:
sigma, sigma_err, mu, mu_err = tuple(df_[["sigma", "sigma_err", "mu", "mu_err"]].values.T)

Ausgehend davon kann eine passende Beschreibung der Breite $\sigma_{G}(m_{\mathrm{H}})$ gefunden werden.

In [ ]:
# your erstimation model and fit goes here

def sigma_model(x, *args):
    """
    Model to for sigma parameterization
    
    """
    return x

## Likelihood - Quotienten - Test

Für die Auswahl zwischen den unterschiedlcihen Simulationen wird explizit der Likelihood-Quotienten-Test verwendet.
Hierzu wird die Nullhypothese $H_0$, die nur dem Untergrund entspricht, und die Gegenhypothese $H_1$, die dem Untergrund und einem Signal entspricht, verglichen.

In die (extended) Likelihood der Hypothese $H_0$ fließt die Parametrisierung des Untergrundes $f_{\mathrm{U}}(x|\theta_{\mathrm{U}})$ ein wobei f_{\mathrm{U}} das vorher festgelegte Modell aus der Untergrundparametrisierung mit den optimalten Parametern $\theta_{\mathrm{U}}$ ist.

Die für die Gegenhypothese verwendete Verteilung lässt sich beschreiben durch: $$f_{\mu\mathrm{S+B}}(m_{4\ell}|\mu) = \mu f_{\mathrm{S}}(m_{4\ell}|\theta_{\mathrm{S}}) + f_{\mathrm{U}}(m_{4\ell}|\theta_{\mathrm{U}}) \, , \\ \int_{\Omega} f_{\mu\mathrm{S+B}}(m_{4\ell}|\mu) \mathrm{d}m_{4\ell} = 1 $$ $f_{\mathrm{S}}(x|\theta_{\mathrm{S}})$ ist für diesen Fall eine Gaussverteilung mit festgehaltenen Werten für den Scherpunkt und die Breite. Für den Schätzer $\mu$ gilt: $$\mu \geq 0 \, .$$ Diese Einschränkung fürt zu einer vernachlässigbaren Verzerrung und wird aus physikalischen Überlegungen eingeführt, da in der Verteilung der Invarianten massen ein Überschuss an Ereignissen erwartet wird, wenn ein Higgs Boson vorhanden sein sollte.

Damit kann die zu betrachtende Größe - das Verhältnis aus den Likelihoods beider Hypothesen - gebildet werden. Für die numerische stabilität wird der negative Logarithmus davon gebildet. Fer Faktor zwei ist für die spätere asymthotische Form notwendig.

$$ q_0 = -2 \ln \left( \frac{\mathcal{L}(x|\mu = \hat{\mu})}{\mathcal{L}(x|\mu = 0)} \right) \, .$$ 
$\hat{\mu}$ ist hierbei der ideale Schätzer für eine feste Masse und Breite der Signalhypothese.

In [ ]:
lower_bound, higher_bound = 106, 151
mass_array = np.linspace(lower_bound, higher_bound, 100)
model_arguments = tuple([])
sigma_array = sigma_model(mass_array, *model_arguments)

In [ ]:
# your implementation of QLT goes here.

Die asymthotische Form der Wahrscheinlichkeitsdichte von $q_0$ wenn kein Signal erwartet wird lässt sich beschreiben durch:
$$ f(q_0|\hat{\mu}=0) \rightarrow \frac{1}{2}\left( \delta (q_0) + \chi^2_{1} \right) \, .$$
Daraus lässt sich der $p_0$ Wert berechnen nach:
$$ p_0 = \int\limits_{q_{0_{\mathrm{obs}}}}^{\infty} \mathrm{d}q_0 f(q_0|\hat{\mu}=0) \, . $$
Aus diesem lässt sich die Signifikanz abschätzend berechnen durch: $$Z = \Phi^{-1}(1-p_0) = \sqrt{q_{0_{\mathrm{obs}}}} $$. In diesem Fall sollte diese jedoch nur als eine erste Abschätzung betrachtet werden da die eingehende Parametrisierung des Signals ungenau ist.
Ebenso wird an dieser Stelle die Systematik nicht betrachtet, es fließt nur die statistische Unsicherheit in die Berechnung von $p_0$ bzw.  $Z$ ein.

Das Abspeichern  geschieht in Form von einem Pandas Dataframe.

In [ ]:
q0, p0 = np.zeros(len(sigma_array)), np.zeros(len(sigma_array))
df = pd.DataFrame(data=np.array([mass_array, sigma_array, q0, p0, np.sqrt(q0)]).T,
                           columns=["mass", "gauss_sigma", "q0", "p0", "sqrt_q0"])

Die Darstellung der Ergebnisse kann entweder von Studenten implementiert werden oder eine vordefinierte FUnktion verwendet werden.

<ins> Version 1: <ins>

In [ ]:
# your Plot goes here

<ins> Version 2: <ins>

In [ ]:
from include.stattest.StatTestInit import StatTestInit

p0_plot = StatTestInit()
p0_plot.plot_p0_estimation(plot_=df, to_file_=False)


# Parametrisierung der Signal MC
----------------------

Nach der Wahl der bestpassenden Signalhypothese soll die Verteilung genauer parametrisiert werden. Dies geschieht analog zu der Untergrundparametrisierung. Zur Auswahl stehen den Studenten:  
 - Gaussverteilung
 - Cauchyverteilung
 - Voigt-Profil
 - Single - Sided - Crystal - Ball
 - Double - Sided - Crystal - Ball  

Die beiden letzen müssen die Studenten nicht eigenständig implementieren und werden als Methoden ("DSCB" und "SSCB") in der Klasse XX zur verfügung gestellt.

In [ ]:
from include.parameterization.SignalFunction import SignalFunction as SF

Variante:

 1. <ins>Die Daten aus dem Histogramm werden roh ausgegeben.</ins>  
    Die skalierten Unsicherheiten für die Bineinträge müssen separat berechnet werden. Zudem muss die Skalierung des
    Histogramms vorgenommen werden.  
    Das Zugreifen auf die einzelnen Teile kann über dne Attribut data geschehen, wobei das ein geschachteltes PYTHON
    dict ist. Die erste Ebene enthält die drei einzelnen Kanäle. In jedem der Kanäle ist anschließend die unskalierten
    Histogrammeinträge, der Korrekturfaktor und die Daten, die für die Histogrammbildung verwendet wurden gespeichert.

In [ ]:
raw_mc_data = HDG(bins=bins, hist_range=hist_range, 
           info=info, mc_dir=mc_dir, ru_dir=ru_dir).get_mc_raw(tag="signal")
print(list(raw_mc_data.data.keys()))
raw_mc_data.data["mc_track_H_ZZ_4el_2012"]

In [ ]:
# your functions goes here
# your fit goes here

 2. <ins>Die Daten aus dem Histogramm werden zusammengefasst ausgegeben</ins>  
     und die Unsicherheit bereits berechnet

In [ ]:
from include.fits.McFitInit import McFitInit

In [ ]:
format_data = McFitInit(bins=bins, hist_range=hist_range, tag="background", mc_dir=mc_dir, ru_dir=ru_dir)
bins, hist_range, hist_entries, combined_hist_entries_errors = format_data.variables_for_hist_fit

In [ ]:
# your functions goes here
# your fit goes here

Hier können die Studenten die oben bereits vervollständigte Klasse verwenden, sofern sie diese Variante gewählt haben.

# Bestimmung der statistischen Signifikanz
----------------------

Nach der erfolgreichen parametrisierung des Untergrundes und des Signals kann die Signifikanz des Signals bestimmt werden. Hierzu wird folgende Parametrisierung gewählt:
$$ f(x, \alpha_\mathrm{S}) = \alpha_\mathrm{S} f_{\mathrm{S}}(x|\theta_{\mathrm{S}}) + (1 - \alpha_\mathrm{S}) f_{\mathrm{U}}(x|\theta_{\mathrm{U}}) \, .$$

Der optimale Parameter $\alpha_{\mathrm{S}}$ entspricht der Signalstärke dieser Messung. Diese Bestimmung erfolgt mithilfe der eigenständig zu implementierenden $-2\ln\mathcal{L(\alpha_{\mathrm{S}})}$ - Funktion. Die zuvor bestimmten Parameter $\theta_{\mathrm{S}}$ und $\theta_{\mathrm{U}}$ der Wahrscheinlichkeitsdichte des Signals bzw. Untergrundes werden für die Bestimmung festgehalten (profiled Likelihood).

In [ ]:
# your implementation of -2lnL goes here
# Hint: it is clever to implement an 2D variant. The 1D variant can then be trivially derived.
def nll(alpha, mass):
    return np.zeros(len(alpha))

Die Betrachtung des Funktionswertes $-2\ln\mathcal{L}(\alpha_{\mathrm{S}} = 0)$ soll hierbei hervorgehoben werden. Dieser Entspricht nähmlich der Annahme, dass in der vorhandenen Messung kein Signalanteil vorhanden ist und die Messung somit durch die Nullhypothese (nur Untergrund) erklärt werden kann. In asymthotischen Form lässt sich deshalb analog zu dem im Kapitel zur Parametrisierung der Signal MC Ausdruck die Signifikanz bestimmen mit: $$ Z = \sqrt{\frac{-2\ln\mathcal{L}(\alpha_{\mathrm{S}} = 0)}{\mathcal{L}_{\mathrm{min}}}} \, .$$

## 1D profiled Likelihood

Die eindimensionale Variante variiert nur die Signalstärke. Die Masse wird nach der Parametrisierung festgehalten und nicht mehr variiert.

In [ ]:
# your minimization goes here
# Hint: you may use allready implemented minimizer as scipy or iminuit (iminuit >> scipy)

In [ ]:
# your plot goes here
# you can use a predefined function

In [ ]:
from include.stattest.PlotHelper import PlotHelper
import matplotlib.pyplot as plt
from mpl_toolkits.axes_grid1.inset_locator import inset_axes, mark_inset

In [ ]:
alpha_ = np.linspace(0.0, 0.8, 1000)
nll_ = nll(alpha=alpha_, mass=125)

# as example
nll_ = 150*(alpha_ - 0.4) ** 2

z_ = np.sqrt(nll_[0])

# as example
def plot_1d(alpha_=alpha_, nll_=nll_, z_=np.sqrt(nll_[0])):
    fig, ax = plt.subplots(1,1)
    PlotHelper.plot_nll_scan_main_window(ax_obj_=ax, x_=alpha_, y_=nll_, z_=z_)
    ax_in = inset_axes(ax, width='50%', height='40%', loc="upper right")
    PlotHelper.plot_nll_scan_inside_window(ax_obj_=ax_in, x_=alpha_, y_=nll_, 
                                           x_ticks_=PlotHelper.get_error_points_from_nll(x_=alpha_, nll_=nll_))
    mark_inset(ax, ax_in, loc1=2, loc2=4, fc="none", ec="grey", lw=0.5, alpha=0.5)

plot_1d()

## 2D profiled Likelihood - Massenbestimmung

Abschließend wird auch die Masse bestimmt. Dies geschieht durch die zweidimensionale Variation - Masse und Signalstärke

In [ ]:
# your minimization goes here
# you might implement a new nll function if you do not yet have a two-dimensional nll function

In [ ]:
# your plot goes here
# you can use a predefined function

# Ergebniszusammenfassung
----------------------

Es wurde eine Signifikanz eines higgsartigen Bosons und seine Masse bestimmt. Durch die Berücksichtigung der Ergebnisse aus den anderen Kanälen kann darauf geschlossen werden, dass es sich um das Higgs-Boson handelt.

Sofern noch zusätzliche Likelihood aus dem ATLAS Experiment vorliegt kann dieser mit der vorhandenen Liklihood zusammengefasst werden.